<h1> <center> Amazon Fake Reviews </center> </h1>

In [1]:
#!pip install langdetect
#!pip install language_check

In [2]:
#General purpose
import os
#Data handling
import pandas as pd
import numpy as np
import datetime
#Data Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
#Modelling 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from langdetect import detect #to filter non-english reviews.
import language_check #to detect errors
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
## nltk downloads
# nltk.download('punkt') # At first you have to download these nltk packages.
# nltk.download('stopwords')
# nltk.download('wordnet')

In [7]:
#Load dataset
df=pd.read_csv('../data/exports/test_appliances.csv')
df.head()

,asin,reviewDate,reviewerID,reviewerName,reviewTitle,productGradeByReview,reviewText,reviewUpvotes,reviewHasImage,reviewIsVerified,...,productTitle,productDescription,productFeatures,productHasImage,productCategory,producBrand,productRank,productYear,productMonth,productDay
0,B00009W3I4,2013-08-22,A34A1UP40713F8,James. Backus,Great product,5,I like this as a vent as well as something tha...,0,0,1,...,HEARTLAND 21000 Energy Saving Dryer Vent Closure,Replace your old vent with the Heartland Natur...,EASY-TO-INSTALL Dryer vent closure with hassl...,1.0,Tools & Home Improvement,HEARTLAND,5633.0,2003.0,5.0,14.0
1,B00009W3PA,2016-02-08,A1AHW6I678O6F2,kevin.,Five Stars,5,good item,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0
2,B00009W3PA,2015-08-05,A8R48NKTGCJDQ,CDBrannom,Five Stars,5,Fit my new LG dryer perfectly.,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0
3,B00009W3PA,2015-04-24,AR3OHHHW01A8E,Calvin E Reames,Perfect size,5,Good value for electric dryers,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0
4,B00009W3PA,2015-03-21,A2CIEGHZ7L1WWR,albert j. kong,Five Stars,5,Price and delivery was excellent.,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0


In [8]:
df.columns

Index(['asin', 'reviewDate', 'reviewerID', 'reviewerName', 'reviewTitle',
       'productGradeByReview', 'reviewText', 'reviewUpvotes', 'reviewHasImage',
       'reviewIsVerified', 'reviewYear', 'reviewMonth', 'reviewDay',
       'productDate', 'productTitle', 'productDescription', 'productFeatures',
       'productHasImage', 'productCategory', 'producBrand', 'productRank',
       'productYear', 'productMonth', 'productDay'],
      dtype='object')

# Useful functions

In [14]:
#Print review
def print_review(df,i):
    print('-------------------')
    print('Review #{}'.format(i))
    print('Title: {},\nReview: {},\nReviewer ID: {},\nVerified: {}'
          .format(df['reviewTitle'][i], df['reviewText'][i], df['reviewerID'][i], df['reviewIsVerified'][i]))
    

In [29]:
#Print product
def print_product(df,i):
    print('-------------------')
    print('Product: {}'.format(df['asin'][i]))
    print('Name: {},\nCategory: {},\nRank: {},\nBrand: {}\nDescription: {}'
         .format(df['productTitle'][i],df['productCategory'][i],df['productRank'][i],df['productBrand'][i],df['productDescription'][i]))
#ToDo
#Modify to be able to get and print a product by its ID

In [16]:
#Cosine Similarity
def cosine_sim_vectors(vec1,vec2):
    vec1=vec1.reshape(1,-1)
    vec2=vec2.reshape(1,-1)
    
    return cosine_similarity(vec1,vec2)[0][0]

# Data Analysis

In [11]:
df.dtypes

asin                     object
reviewDate               object
reviewerID               object
reviewerName             object
reviewTitle              object
productGradeByReview      int64
reviewText               object
reviewUpvotes             int64
reviewHasImage            int64
reviewIsVerified          int64
reviewYear                int64
reviewMonth               int64
reviewDay                 int64
productDate              object
productTitle             object
productDescription       object
productFeatures          object
productHasImage         float64
productCategory          object
producBrand              object
productRank             float64
productYear             float64
productMonth            float64
productDay              float64
dtype: object

* reviewDate and productDate should be changed to datetime
* The categorical variables (productGradeByReview, reviewUpvotes*, reviewHasImage, reviewIsVerified could be change temporarily to float to use the Pandas Describe function (cf cell below)

In [10]:
df.describe(include='all')

,asin,reviewDate,reviewerID,reviewerName,reviewTitle,productGradeByReview,reviewText,reviewUpvotes,reviewHasImage,reviewIsVerified,...,productTitle,productDescription,productFeatures,productHasImage,productCategory,producBrand,productRank,productYear,productMonth,productDay
count,2277,2277,2277,2277,2277,2277.000000,2277,2277.000000,2277.000000,2277.000000,...,2275,190,2245,2275.000000,2275,2275,2.274000e+03,2275.000000,2275.000000,2275.000000
unique,48,99,47,47,96,NaN,144,NaN,NaN,NaN,...,46,41,40,NaN,5,26,NaN,NaN,NaN,NaN
top,B0006GVNOA,2016-12-06,A1WD61B0C3KQZB,CP,Does what it needs to,NaN,At first this contraption was a little confusi...,NaN,NaN,NaN,...,Gardus RLE202 LintEater Rotary Dryer Vent Clea...,Constructed of 4-ply aluminum foil laminate. F...,Most valuable tool you can own to ensure the s...,NaN,Tools &amp; Home Improvement,LintEater,NaN,NaN,NaN,NaN
freq,2040,412,206,206,206,NaN,206,NaN,NaN,NaN,...,2040,12,2064,NaN,2040,2064,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,4.499780,NaN,221.378568,0.363636,0.987264,...,NaN,NaN,NaN,0.952527,NaN,NaN,6.880796e+03,2001.767473,4.291868,7.040879
std,NaN,NaN,NaN,NaN,NaN,0.837723,NaN,542.987888,0.481151,0.112158,...,NaN,NaN,NaN,0.212694,NaN,NaN,4.348922e+04,72.809410,1.185184,4.175630
min,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,0.000000,NaN,NaN,1.000000e+00,-1.000000,-1.000000,-1.000000
25%,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,5.000000,0.000000,1.000000,...,NaN,NaN,NaN,1.000000,NaN,NaN,1.976000e+03,2004.000000,4.000000,6.000000
50%,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,8.000000,0.000000,1.000000,...,NaN,NaN,NaN,1.000000,NaN,NaN,1.976000e+03,2004.000000,4.000000,6.000000
75%,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,165.000000,1.000000,1.000000,...,NaN,NaN,NaN,1.000000,NaN,NaN,1.976000e+03,2004.000000,4.000000,6.000000


* The dataset contains 2277 reviews for 48 different products and from 47 different reviewers
* The categorical variables are not analyzed properly with the describe method => A solution would be to convert them to float
* 75% of the reviews assign a grade of 5 to the products
* We know from before that most of the reviews are verified with a ratio around 1000/1.


In [20]:
#Confirming that we have reviews for only 48 products
df['asin'].value_counts().shape

(48,)

In [21]:
df['asin'].value_counts().head()

B0006GVNOA    2040
B0014CN8Y8      24
B00JGTUQJ6      13
B0015UGPWQ      12
B004XLDE5A       8
Name: asin, dtype: int64

In [30]:
#Let's examine this product
temp=df[df['asin']=='B0006GVNOA'].head(1).reset_index()
print_product(temp,0)

-------------------
Product: B0006GVNOA
Name: Gardus RLE202 LintEater Rotary Dryer Vent Cleaning System,
Category: Tools &amp; Home Improvement,
Rank: 1976.0,
Brand: LintEater
Description: nan


# Data Processing